In [114]:
from wit import Wit
import pandas as pd
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from bs4 import BeautifulSoup
from collections import defaultdict
import requests
%matplotlib inline

# 1.Selected_comments_with_witai_sentiment_results
## import comment with side effects for query

In [1]:
SE_tem_l400 = pd.read_csv("wit_ai_sentment/SE_tem_l400.csv",index_col=0,header=None)[1]
SE_tem_l400 = SE_tem_l400.iloc[3:].astype(str)
idxs = list(SE_tem_l400.index)

## query against wit.ai

In [6]:
access_token='TPGKFHFWL6UWQ2MNBTX7LBVKZP4FNYWN'
client = Wit(access_token)

sentments = {}

test = SE_tem_l400#.iloc[:10]

def sent_one_message(idx):
    b = SE_tem_l400.loc[idx]
    try:
        res = client.message(b)#['entities']
        return [idx,res]
    
    except Exception:
        pass
    

pool_size = 6
pool = multiprocessing.Pool(pool_size)
test = pool.map(sent_one_message, idxs)

Process ForkPoolWorker-22:
Process ForkPoolWorker-23:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Traceback (most recent call last):
  File "/Users/jzz0026/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/jzz0026/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jzz0026/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jzz0026/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/jzz0026/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/jzz0026

## Sort results

In [20]:
test2= [a for a in test if a != None]
first_quest = pd.DataFrame(test2).set_index(0)
#first_quest.to_csv("SE_tem_l400_first_request.scv")

In [62]:
first_quest[2] = first_quest[1].apply(lambda x : x['entities'])
first_quest2 = first_quest[first_quest[2] != {}]
first_quest2["confidence"] = first_quest2[2].apply(lambda x : x['sentiment'][0]['confidence'])
first_quest2['value'] = first_quest2[2].apply(lambda x : x['sentiment'][0]['value'])
first_quest2['text'] = first_quest2[1].apply(lambda x: x['_text'])# = 

first_quest2 = first_quest2[['text','confidence','value']]
first_quest2.head()


/Users/jzz0026/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jzz0026/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/jzz0026/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,text,confidence,value
0,,,
206461,"It has no side effect, I take it in combinatio...",0.859484,neutral
92703,"I used to take another oral contraceptive, whi...",0.669752,positive
102654,Only side effect is I like to eat a lot,0.544129,neutral
75612,"Best thing, no side effects",0.801017,positive
81890,It has since subsided and the only side effect...,0.501730,neutral


In [161]:
#first_quest2.to_csv("wit_ai_sentment/SE_tem_l400_first_request.ccv")

In [113]:
print("Total results:",len(first_quest2))
first_quest2.groupby('value').size()

Total results: 30689


value
negative    16765
neutral     12010
positive     1914
dtype: int64

In [111]:
first_quest2[first_quest2['value'] == 'negative'].sample(1)['text'].iloc[0]

"Don't be scared by the minor side effects, because they are not greater then a baby"

# 2. Train the model using only these comment with side effects 

In [129]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud

from sklearn.svm import SVC

## only need to remove punctuation and stemize
stemmer = SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [130]:
## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

In [131]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0)
df = rm_sym(df)


test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0)
test = rm_sym(test)

In [134]:
train = pd.concat([df.loc[first_quest2.index], first_quest2],1)
## Generate table of words with their counts
con_vec = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
X_train_p = con_vec.fit_transform(train['review'])
y_train = train['rating_cate']


/Users/jzz0026/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [143]:
negnornot = (train['value'] == "negative").astype(int)
negnornot.shape

(30689,)

In [147]:
from scipy.sparse import hstack

X_train_dtm = hstack((X_train_p,np.array(negnornot)[:,None]))
X_train_dtm

<30689x24463 sparse matrix of type '<class 'numpy.float64'>'
	with 1167174 stored elements in COOrdinate format>

## Logistic regression

In [157]:
lr = LogisticRegression(penalty='l1',multi_class='auto',solver='saga')
#lr.fit(X,target_3.iloc[:,0])

cv_score = cross_val_score(lr,X_train_dtm,y_train,scoring='accuracy',cv=3,n_jobs=-1)
cv_score

array([0.7862379 , 0.79061584, 0.78607743])

### RandomForestClassifier

In [158]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc_cv_score = cross_val_score(rfc,X_train_dtm,y_train,scoring='accuracy',cv=3,n_jobs=-1)
rfc_cv_score

array([0.83647737, 0.84154448, 0.83525616])

In [159]:
## n_estimators=300
## array([0.83647737, 0.84154448, 0.83525616])

## n_estimators=200
## array([0.83686834, 0.84183773, 0.83633164])


### Gradient boosting classifier

In [156]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=300)
gbc_cv_score = cross_val_score(gbc,X_train_dtm,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.7762682 , 0.77800587, 0.77620258])

### SVM

#### poly

In [150]:
svm_poly = SVC(kernel='poly')
gbc_cv_score = cross_val_score(svm_poly,X_train_dtm,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.71342   , 0.71348974, 0.71353148])

#### linear

In [148]:
svm_lin = SVC(kernel='linear')
gbc_cv_score = cross_val_score(svm_lin,X_train_dtm,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.79962858, 0.80371457, 0.80201408])